In [9]:
import ndjson
import cv2
import os
import pandas as pd
import numpy as np

# Open and read the NDJSON file
with open('D:\\Manager-angry-meter\\video-input\\videos.ndjson', 'r') as f:
    data = ndjson.load(f)


print(f"Combined records: {len(data)}")


Combined records: 69


In [10]:
df_meta = pd.DataFrame(data)
df_meta.drop(columns=['media_attributes'], inplace=True)
df_meta['data_row'] = df_meta['data_row'].astype(str).str[8:37]
df_meta.rename(columns={'data_row': 'id'}, inplace=True)
df_meta.rename(columns={'metadata_fields': 'emotion'}, inplace=True)
df_meta.head()

,id,emotion
0,cmhpke9p4cjbs0782sob057xx.mp4,[{'value': 'Surprised'}]
1,cmhpke9p4cjbt0782363n2x0u.mp4,[{'value': 'Surprised'}]
2,cmhpke9p4cjbu0782assgc5x9.mp4,[{'value': 'Surprised'}]
3,cmhpke9p4cjbv0782ah5bfrg4.mp4,[{'value': 'Surprised'}]
4,cmhpke9p4cjbw0782wkp7yx3s.mp4,[{'value': 'Surprised'}]


In [11]:
# --- ROBUST DATA CLEANING ---
def clean_emotion_col(x):
    # 1. If it's a list (e.g., ['angry']), grab the first item
    if isinstance(x, list):
        if len(x) > 0:
            x = x[0]
        else:
            return "unknown"

    # 2. If it's a dictionary (e.g., {'label': 'angry'}), grab the value
    if isinstance(x, dict):
        # Try common keys, or just grab the first value found
        x = x.get('label', x.get('emotion', list(x.values())[0]))

    # 3. Convert whatever is left to a string and lower-case it
    text = str(x).lower()
    
    # 4. Remove any leftover garbage characters
    text = text.replace("'", "").replace('"', "").strip()
    return text

# Apply the fix
print("Cleaning data...")
df_meta['emotion'] = df_meta['emotion'].apply(clean_emotion_col)
# --- VERIFY ---
print("\nUnique emotions found:")
print(df_meta['emotion'].unique())

Cleaning data...

Unique emotions found:
['surprised' 'sad' 'neutral' 'happy' 'disgust' 'angry']


In [14]:
import cv2
import os
import pandas as pd
import numpy as np

# --- CONFIGURATION ---
video_folder = "D:\\Manager-angry-meter\\data\\videos"
output_folder = "data/processed_frames_HD" 
img_size = (128, 128) 

# --- FIX IS HERE ---
# 1. Define the path
cascade_path = r"C:\Users\PC\anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml"

# 2. Check if file exists (Crucial because Windows paths can be tricky)
if not os.path.exists(cascade_path):
    print(f"❌ ERROR: Cannot find the Cascade XML at: {cascade_path}")
    print("   Please check the path again.")
    exit()

# 3. Actually LOAD the classifier object
face_cascade = cv2.CascadeClassifier(cascade_path)

# Verify it loaded correctly
if face_cascade.empty():
    print("❌ ERROR: Cascade file found but failed to load. Is it corrupt?")
    exit()
else:
    print(f"✅ Face Detector loaded successfully from: {cascade_path}")


# --- START HARVESTING ---
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print(f"🚜 Starting HD Harvest from {len(df_meta)} videos...")

count = 0
faces_found_count = 0

for index, row in df_meta.iterrows():
    video_filename = row['id']
    emotion_label = row['emotion']
    
    # Check extension
    if not video_filename.endswith('.mp4'):
        video_filename += ".mp4"
        
    video_path = os.path.join(video_folder, video_filename)
    
    # Skip if video file missing
    if not os.path.exists(video_path):
        continue

    # Create Emotion Folder
    label_path = os.path.join(output_folder, emotion_label)
    os.makedirs(label_path, exist_ok=True)
    
    cap = cv2.VideoCapture(video_path)
    frame_id = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # 1. Convert to Gray (Face detection needs grayscale)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # 2. Detect Faces
        # scaleFactor=1.1 -> Scans image at different sizes (10% increments)
        # minNeighbors=4  -> Higher = fewer false positives, but might miss faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
        
        # 3. If a face is found, crop it
        if len(faces) > 0:
            for (x, y, w, h) in faces:
                # Crop: [y_start:y_end, x_start:x_end]
                face_img = frame[y:y+h, x:x+w]
                
                try:
                    # High Quality Resize
                    face_resized = cv2.resize(face_img, img_size, interpolation=cv2.INTER_AREA)
                    
                    # Save
                    clean_name = video_filename.replace('.mp4', '')
                    save_name = f"{clean_name}_f{frame_id}.jpg"
                    cv2.imwrite(os.path.join(label_path, save_name), face_resized)
                    
                    faces_found_count += 1
                except Exception as e:
                    pass # Skip if resize crashes (rare)
                
                # Only take the first face to avoid duplicates/crowd
                break 
        
        frame_id += 1
        count += 1

    cap.release()

print(f"✅ HD Harvest Complete!")
print(f"   Scanned {count} total frames.")
print(f"   Saved {faces_found_count} high-quality face images to '{output_folder}'")

✅ Face Detector loaded successfully from: C:\Users\PC\anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml
🚜 Starting HD Harvest from 69 videos...
✅ HD Harvest Complete!
   Scanned 4677 total frames.
   Saved 4669 high-quality face images to 'data/processed_frames_HD'


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = "data/processed_frames_HD" 
img_height, img_width = 128, 128 

datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
)

print(f"📂 Loading Training Data from {dataset_path}...")
train_ds = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

print(f"📂 Loading Validation Data from {dataset_path}...")
val_ds = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    
    Dense(64, activation='relu'),
    Dropout(0.3),
    
    Dense(train_ds.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("🚀 Starting Training on HD Data...")
history = model.fit(train_ds, epochs=15, validation_data=val_ds)

📂 Loading Training Data from data/processed_frames_HD...
Found 3863 images belonging to 6 classes.
📂 Loading Validation Data from data/processed_frames_HD...
Found 964 images belonging to 6 classes.


c:\Users\PC\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


🚀 Starting Training on HD Data...
Epoch 1/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 33s 265ms/step - accuracy: 0.3508 - loss: 1.5737 - val_accuracy: 0.3693 - val_loss: 1.4441
Epoch 2/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 12s 95ms/step - accuracy: 0.6032 - loss: 1.0307 - val_accuracy: 0.6214 - val_loss: 1.1729
Epoch 3/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 12s 103ms/step - accuracy: 0.7468 - loss: 0.6686 - val_accuracy: 0.7106 - val_loss: 1.1385
Epoch 4/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 12s 100ms/step - accuracy: 0.8123 - loss: 0.5074 - val_accuracy: 0.6276 - val_loss: 1.0377
Epoch 5/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 12s 99ms/step - accuracy: 0.8514 - loss: 0.4052 - val_accuracy: 0.5716 - val_loss: 1.2139
Epoch 6/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 12s 103ms/step - accuracy: 0.8796 - loss: 0.3317 - val_accuracy: 0.6649 - val_loss: 1.0316
Epoch 7/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 13s 104ms/step - accuracy: 0.8858 - loss: 0.3216 - val_accuracy: 0.6867 - val_loss: 1.0248
Epoch 8/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 13s 104ms/s

In [6]:
import pandas as pd
import ndjson

# Open and read the NDJSON file
with open('Audio.ndjson', 'r') as f:
    data_audio = ndjson.load(f)


print(f"Combined records: {len(data_audio)}")



Combined records: 50


In [7]:
df_meta_audio = pd.DataFrame(data_audio)
df_meta_audio.drop(columns=['media_attributes'], inplace=True)
df_meta_audio['data_row'] = df_meta_audio['data_row'].astype(str).str[8:37]
df_meta_audio.rename(columns={'data_row': 'id'}, inplace=True)
df_meta_audio.rename(columns={'metadata_fields': 'emotion'}, inplace=True)
df_meta_audio.head()

,id,emotion
0,cmi6kb42m14t80768b3k66mu7.WAV,[{'value': 'Angry'}]
1,cmi6kb42m14t907681xw39xws.WAV,[{'value': 'Angry'}]
2,cmi6kb42m14ta0768107er0vz.WAV,[{'value': 'Angry'}]
3,cmi6kb42m14tb0768roy88bhh.WAV,[{'value': 'Angry'}]
4,cmi6kb42m14tc076895ghj9du.WAV,[{'value': 'Angry'}]


In [8]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU Detected: {gpus}")
    print("TensorFlow is using the GPU.")
else:
    print("❌ No GPU detected. Running on CPU.")

2025-12-22 04:43:15.829161: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-22 04:43:15.882313: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-22 04:43:16.881458: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/adham/gpu_env/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureW

✅ GPU Detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow is using the GPU.


In [9]:
import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
from transformers import Wav2Vec2Processor, TFWav2Vec2Model
import tensorflow as tf
import joblib

# --- CONFIGURATION ---
video_folder = "data"
MODEL_NAME = "facebook/wav2vec2-base-960h" # The Industry Standard for Speech
MAX_DURATION = 3 # seconds

# 1. LOAD PRETRAINED MODEL (The "Brain")
print("📥 Loading Pretrained Wav2Vec2 Model...")
processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)
# We load the TensorFlow version of the model
# output_hidden_states=True lets us peek inside the "brain"
# Force it to use standard weights to avoid the iteration error
base_model = TFWav2Vec2Model.from_pretrained(MODEL_NAME, use_safetensors=False)
def extract_deep_features(path):
    try:
        # A. Load Audio (16kHz is mandatory for Wav2Vec2)
        audio, sr = librosa.load(path, sr=16000, duration=MAX_DURATION)
        
        # B. Pad/Truncate to ensure consistent length
        # 3 seconds * 16000 Hz = 48000 samples
        target_len = 16000 * MAX_DURATION
        if len(audio) < target_len:
            audio = np.pad(audio, (0, target_len - len(audio)))
        else:
            audio = audio[:target_len]

        # C. Prepare Input for the Model
        inputs = processor(audio, sampling_rate=16000, return_tensors="tf")
        
        # D. Run through Pretrained Model
        outputs = base_model(inputs.input_values)
        
        # E. Get the "Embeddings" (The last hidden state)
        # Shape: (1, Sequence_Length, 768) -> We average it to get (768,)
        hidden_states = outputs.last_hidden_state
        features = tf.reduce_mean(hidden_states, axis=1) 
        
        return features.numpy().flatten()
        
    except Exception as e:
        return None

# --- HARVEST LOOP ---
deep_features = []
labels = []

print(f"🧠 Extracting Deep Features from {len(df_meta_audio)} videos...")

for index, row in tqdm(df_meta_audio.iterrows(), total=len(df_meta_audio)):
    filename = row['id']
    path = os.path.join(video_folder, filename)
    
    if os.path.exists(path):
        feats = extract_deep_features(path)
        if feats is not None:
            deep_features.append(feats)
            labels.append(row['emotion'])

# Save to disk so you don't have to run this again
X_deep = np.array(deep_features)
print(f"✅ Extraction Complete! Feature Shape: {X_deep.shape}") 
# Shape should be (Num_Videos, 768) -> 768 is the size of Wav2Vec2 embeddings

if len(deep_features) > 0:
    print(f"💾 Saving {len(deep_features)} samples to disk...")
    
    # Save the features (X)
    joblib.dump(X_deep, 'X_deep_features.pkl')
    
    # Save the labels (y)
    joblib.dump(labels, 'y_deep_labels.pkl')
    
    print("✅ Successfully saved 'X_deep_features.pkl' and 'y_deep_labels.pkl'")
else:
    print("⚠️ No features were extracted. Nothing to save.")

📥 Loading Pretrained Wav2Vec2 Model...


I0000 00:00:1766371400.667465   19784 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.

TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some layers from the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing TFWav2Vec2Model: ['lm_head', 'dropout_50']
- This IS expected if you are initializing TFWav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2Model fr

🧠 Extracting Deep Features from 50 videos...


  0%|                                                                                            | 0/50 [00:00<?, ?it/s]TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
2025-12-22 04:43:22.892995: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91700
2025-12-22 04:43:24.051349: I external/local_xla/xla/service/service.cc:163] XLA service 0x41107830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-22 04:43:24.051380: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
I0000 00:00:1766371404.270646   19784 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
100%|███████████████████████████████████████████████████████████████████████████████████

✅ Extraction Complete! Feature Shape: (50, 768)
💾 Saving 50 samples to disk...
✅ Successfully saved 'X_deep_features.pkl' and 'y_deep_labels.pkl'


In [10]:
# --- ROBUST DATA CLEANING ---
def clean_emotion_col(x):
    # 1. If it's a list (e.g., ['angry']), grab the first item
    if isinstance(x, list):
        if len(x) > 0:
            x = x[0]
        else:
            return "unknown"

    # 2. If it's a dictionary (e.g., {'label': 'angry'}), grab the value
    if isinstance(x, dict):
        # Try common keys, or just grab the first value found
        x = x.get('label', x.get('emotion', list(x.values())[0]))

    # 3. Convert whatever is left to a string and lower-case it
    text = str(x).lower()
    
    # 4. Remove any leftover garbage characters
    text = text.replace("'", "").replace('"', "").strip()
    return text
X_deep = np.array(deep_features)
y_deep = pd.DataFrame(labels, columns=['emotion'])
# Apply the fix
print("Cleaning data...")
y_deep['emotion'] = y_deep['emotion'].apply(clean_emotion_col)

print("\nUnique emotions found:")
print(y_deep['emotion'].unique())

Cleaning data...

Unique emotions found:
['angry' 'disgust' 'happy' 'neutral' 'sad' 'surprised']


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 1. Prepare Labels
# ---------------------------------------------------------
# ERROR WAS HERE: You passed 'labels' (dirty) instead of y_deep['emotion'] (clean)
lb = LabelEncoder()
y_final = to_categorical(lb.fit_transform(y_deep['emotion']))
# ---------------------------------------------------------

print(f"✅ Labels encoded. Classes found: {lb.classes_}")

# 2. Split
# Use y_final here, not y_deep (to avoid variable confusion)
X_train, X_test, y_train, y_test = train_test_split(X_deep, y_final, test_size=0.2, random_state=42)

# 3. BUILD YOUR CUSTOM MODEL
model_transfer = Sequential([
    # Input is 768 (The output of Wav2Vec2)
    tf.keras.Input(shape=(768,)),
    
    # Added Layer 1
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    # Added Layer 2
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    # Output Layer
    Dense(y_final.shape[1], activation='softmax')
])

# 4. Train
model_transfer.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("🚀 Training Transfer Learning Model...")
history = model_transfer.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# 5. Save
model_transfer.save('wav2vec2_transfer_model.h5')

✅ Labels encoded. Classes found: ['angry' 'disgust' 'happy' 'neutral' 'sad' 'surprised']
🚀 Training Transfer Learning Model...
Epoch 1/50


2025-12-22 04:43:36.799986: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-22 04:43:37.054878: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-22 04:43:37.054989: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-22 04:43:37.055008: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the 

2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.2750 - loss: 3.0193 - val_accuracy: 0.3000 - val_loss: 1.7976
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.3500 - loss: 2.1280 - val_accuracy: 0.4000 - val_loss: 1.7642
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.4250 - loss: 1.5324 - val_accuracy: 0.3000 - val_loss: 1.7423
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.3500 - loss: 1.7275 - val_accuracy: 0.2000 - val_loss: 1.7227
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5500 - loss: 1.3293 - val_accuracy: 0.3000 - val_loss: 1.7063
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5000 - loss: 1.1460 - val_accuracy: 0.4000 - val_loss: 1.6938
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5000 - loss: 1.5558 - val_accuracy: 0.4000 - val_loss: 1.6872
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6000 - loss: 1.0446 - val_accuracy: 0.4000 - val_loss: 1.6847
Epoch 9/50
2

In [12]:
import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
from transformers import Wav2Vec2Processor, TFWav2Vec2Model
import tensorflow as tf
import joblib

# --- CONFIGURATION ---
ROOT_DIR = r"kaggle-data/audio"  # Your data folder
MODEL_NAME = "facebook/wav2vec2-base-960h"
MAX_DURATION = 3 # seconds

# --- 0. DEFINE EMOTION MAP (RAVDESS Standard) ---
# Based on the filename format "03-01-05..."
EMOTION_MAP = {
    '01': 'neutral',
    '02': 'neutral',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# --- 1. LOAD PRETRAINED MODEL ---
print("📥 Loading Pretrained Wav2Vec2 Model...")
processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)
base_model = TFWav2Vec2Model.from_pretrained(MODEL_NAME, use_safetensors=False)

def extract_deep_features(path):
    try:
        # A. Load Audio (16kHz is mandatory for Wav2Vec2)
        audio, sr = librosa.load(path, sr=16000, duration=MAX_DURATION)
        
        # B. Pad/Truncate
        target_len = 16000 * MAX_DURATION
        if len(audio) < target_len:
            audio = np.pad(audio, (0, target_len - len(audio)))
        else:
            audio = audio[:target_len]

        # C. Process
        inputs = processor(audio, sampling_rate=16000, return_tensors="tf")
        outputs = base_model(inputs.input_values)
        
        # D. Average Embeddings
        hidden_states = outputs.last_hidden_state
        features = tf.reduce_mean(hidden_states, axis=1) 
        
        return features.numpy().flatten()
        
    except Exception as e:
        print(f"Error reading {path}: {e}")
        return None

# --- 2. HARVEST LOOP (Refactored) ---
deep_features = []
labels = []
filenames_log = [] # Optional: keep track of which file produced which feature

print(f"📂 Scanning recursive directories in: {ROOT_DIR}")
print("🧠 Extracting features...")

# We use os.walk to find all files recursively
for root, dirs, files in os.walk(ROOT_DIR):
    for filename in files:
        if filename.lower().endswith('.wav'):
            
            # --- A. PARSE EMOTION ---
            # Format: 03-01-05-01-01-01-01.wav
            try:
                parts = filename.split('-')
                if len(parts) >= 3:
                    code = parts[2] # The 3rd part is the emotion
                    emotion_label = EMOTION_MAP.get(code, 'unknown')
                else:
                    emotion_label = 'unknown'
            except:
                emotion_label = 'unknown'

            # Skip if we couldn't identify the emotion
            if emotion_label == 'unknown': 
                continue

            # --- B. EXTRACT FEATURES ---
            file_path = os.path.join(root, filename)
            
            # Print progress every now and then (optional)
            # print(f"Processing: {filename} -> {emotion_label}")

            feats = extract_deep_features(file_path)
            
            if feats is not None:
                deep_features.append(feats)
                labels.append(emotion_label)
                filenames_log.append(filename)

# --- 3. SAVE ---
if len(deep_features) > 0:
    X_deep_kaggle = np.array(deep_features)
    print(f"\n✅ Extraction Complete!")
    print(f"   - Samples: {len(X_deep)}")
    print(f"   - Feature Shape: {X_deep.shape}")
    
    print("💾 Saving to disk...")
    joblib.dump(X_deep_kaggle, 'X_deep_features_kaggle.pkl')
    joblib.dump(labels, 'y_deep_labels_kaggle.pkl')
    print("✅ Saved 'X_deep_features_kaggle.pkl' and 'y_deep_labels_kaggle.pkl'")
else:
    print("⚠️ No features were extracted. Check your folder path or filename format.")

📥 Loading Pretrained Wav2Vec2 Model...



TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some layers from the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing TFWav2Vec2Model: ['lm_head', 'dropout_50']
- This IS expected if you are initializing TFWav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFWav2Vec2Model were initialized from the model checkpoint at facebook/wav2vec2-base-960h.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFWav2Vec2Model for predictions with

📂 Scanning recursive directories in: kaggle-data/audio
🧠 Extracting features...

✅ Extraction Complete!
   - Samples: 50
   - Feature Shape: (50, 768)
💾 Saving to disk...
✅ Saved 'X_deep_features_kaggle.pkl' and 'y_deep_labels_kaggle.pkl'


In [14]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

# --- 1. PREPARE LABELS ---
# We use 'labels_kaggle' (the list) directly. 
# No need for a DataFrame ('y_deep') here unless you want to save it later.
lb = LabelEncoder()
y_final = to_categorical(lb.fit_transform(labels))

print(f"✅ Labels encoded. Classes found: {lb.classes_}")

# --- 2. SPLIT ---
# Updated to use 'X_deep_kaggle'
X_train, X_test, y_train, y_test = train_test_split(
    X_deep_kaggle, 
    y_final, 
    test_size=0.2, 
    random_state=42
)

print(f"📊 Training Data Shape: {X_train.shape}")
print(f"📊 Testing Data Shape: {X_test.shape}")

# --- 3. BUILD YOUR CUSTOM MODEL ---
model_transfer = Sequential([
    # Input is 768 (The output of Wav2Vec2)
    # Using dynamic shape is safer: (X_deep_kaggle.shape[1],)
    tf.keras.Input(shape=(768,)),
    
    # Added Layer 1
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    # Added Layer 2
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    # Output Layer
    Dense(y_final.shape[1], activation='softmax')
])

# --- 4. TRAIN ---
model_transfer.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("🚀 Training Transfer Learning Model...")
history = model_transfer.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# --- 5. SAVE ---
# Updated filename to reflect the Kaggle dataset
model_transfer.save('wav2vec2_transfer_model_kaggle.keras')
print("✅ Model saved as 'wav2vec2_transfer_model_kaggle.keras'")

✅ Labels encoded. Classes found: ['angry' 'disgust' 'fearful' 'happy' 'neutral' 'sad' 'surprised']
📊 Training Data Shape: (384, 768)
📊 Testing Data Shape: (96, 768)
🚀 Training Transfer Learning Model...
Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.1328 - loss: 2.8311 - val_accuracy: 0.1354 - val_loss: 1.9512
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1797 - loss: 2.4157 - val_accuracy: 0.2188 - val_loss: 1.9640
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2005 - loss: 2.3699 - val_accuracy: 0.1979 - val_loss: 1.9501
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1901 - loss: 2.2935 - val_accuracy: 0.2083 - val_loss: 1.9288
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1849 - loss: 2.1791 - val_accuracy: 0.1979 - val_loss: 1.9301
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2396 - loss: 2.0714 - val_accuracy: 0.1562 - val_loss: 1.9255
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0